# Cleaning

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# preprocessing/pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.compose import make_column_selector as selector


# import data
full_data = pd.read_csv("Rail_Equipment_Accident_Incident_Data.csv")
print(full_data.shape)
full_data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (27,43,46,47,48,50,59,61,62,63,64,65,66,67,68,69,71,130,131,136) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(216100, 160)


,Reporting Railroad Code,Reporting Railroad Name,Report Year,Accident Number,PDF Link,Accident Year,Accident Month,Other Railroad Code,Other Railroad Name,Other Accident Number,...,Other Railroad SMT Grouping,Other Parent Railroad Company Code,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Code,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,NS,Norfolk Southern Railway Company,2016.0,120068,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
1,NS,Norfolk Southern Railway Company,2016.0,120068,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
2,CR,Conrail,1981.0,0420001,https://safetydata.fra.dot.gov/Officeofsafety/...,81.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 3,NaN,CRSH,Consolidated Rail Corporation,Not Assigned
3,NS,Norfolk Southern Railway Company,2016.0,120161,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company
4,NS,Norfolk Southern Railway Company,2016.0,120161,https://safetydata.fra.dot.gov/Officeofsafety/...,16.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,NS,Norfolk Southern Railway Company,Norfolk Southern Railway Company


## Dropping Features

In [2]:
features_to_drop = ["Reporting Railroad Code", #code variables/features
                 "PDF Link",
                 "Other Railroad Code",
                 "Maintenance Railroad Code",
                 "Accident Type Code",
                 "State Code",
                 "County Code",
                 "Visibility Code",
                 "Weather Condition Code",
                 "Track Type Code",
                 "Train Direction Code",
                 "Equipment Type Code",
                 "Signalization Code",
                 "Method of Operation Code",
                 "Remote Control Locomotive Code",
                 "Primary Accident Cause",
                 "Contributing Accident Cause Code",
                 "Contributing Accident Cause",
                 "Class Code",
                 "Reporting Parent Railroad Company Code",
                 "Other Parent Railroad Company Code",
                 "Division Code",
                 "Maintenance Parent Railroad Company Code", #identification variables/features
                 "Accident Number",
                 "First Car Initials",
                 "Report Key",
                 "Incident Key",
                 "Train Number",
                 "Maintenance Accident Number",
                 "State Name" #have state abbrev
                 
]

data = full_data.drop(features_to_drop,axis=1)
data.shape

(216100, 130)

In [3]:
# recode "Other ..." to just "Other"
data['Accident Type'] = data['Accident Type'].replace('Other (describe in narrative)','Other')


In [4]:
# data["Accident Cause"].head(5)
s = "JOB SHOVING 8 CARS INTO 121 TRACK DERAILED TWO CARS AND TWO ENGINES JUST EAST OF 121 SWITCH"
# find the column that contains s
data.columns[data.isin([s]).any()]

Index([], dtype='object')

In [5]:
# Dropping additional columns here that we won't initially need
data = data.drop(["Accident Cause"],axis=1)

## Feature Engineering/Cleaning

### Handling NaNs

In [13]:
# Handling NaNs first 
# remove columns with > 50,000 NaNs
# set remaining NaNs to mean for numerical data, encode "NA" for categorical
na_counts = data.isna().sum()
cols_to_drop = na_counts[na_counts > 50000].index.tolist()
data.drop(cols_to_drop, axis=1, inplace=True)

#split data into numerical and categorical data to process

# quanititative features
num_data = data.select_dtypes(include=['float64', 'int64'])
#num_data.fillna(num_data.mean(), inplace=True)
print(num_data.isna().sum().sort_values(ascending=False))

# qualititative features
cat_data = data.select_dtypes(include=['object'])
cat_data.fillna('NA', inplace=True)
cat_data = cat_data.astype(str)
#print(cat_data.isna().sum().sort_values(ascending=False))
#print(num_data.dtypes)
new_dat = pd.concat([num_data,cat_data], axis=1)
new_dat.head()

def before_preprocess(data):
    num_data = data.select_dtypes(include=['float64', 'int64'])
    cat_data = data.select_dtypes(include=['object'])
    cat_data.fillna('NA', inplace=True)
    cat_data = cat_data.astype(str)
    new_dat = pd.concat([num_data,cat_data], axis=1)
    return new_dat



Hours Engineers On Duty                   45952
Firemen On Duty                           40014
Brakemen On Duty                          33774
Conductors On Duty                        27498
Engineers On Duty                         25561
First Car Position                        13357
Train Speed                                   3
Report Year                                   1
Derailed Loaded Passenger Cars                1
Derailed Empty Freight Cars                   1
Derailed Empty Passenger Cars                 1
Derailed Cabooses                             1
Equipment Damage Cost                         1
Track Damage Cost                             1
Total Damage Cost                             1
Railroad Employees Killed                     1
Cabooses                                      1
Railroad Employees Injured                    1
Passengers Killed                             1
Passengers Injured                            1
Others Killed                           

,Report Year,Accident Year,Accident Month,Maintenance Accident Year,Maintenance Accident Month,Day,Hazmat Cars,Hazmat Cars Damaged,Hazmat Released Cars,Persons Evacuated,...,Joint Track Class,Class,Reporting Railroad Class,Reporting Railroad SMT Grouping,Reporting Parent Railroad Company Name,Reporting Railroad Holding Company,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,2016.0,16.0,4.0,16.0,4.0,9.0,0.0,0.0,0.0,0.0,...,1,CL,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
1,2016.0,16.0,4.0,16.0,4.0,9.0,4.0,0.0,0.0,0.0,...,1,CL,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
2,1981.0,81.0,4.0,81.0,4.0,6.0,0.0,0.0,0.0,0.0,...,1,1L,Class 3,NA,Consolidated Rail Corporation,Not Assigned,Class 3,NA,Consolidated Rail Corporation,Not Assigned
3,2016.0,16.0,4.0,16.0,4.0,16.0,0.0,0.0,0.0,0.0,...,1,CL,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
4,2016.0,16.0,4.0,16.0,4.0,16.0,0.0,0.0,0.0,0.0,...,1,CL,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company


### Preprocessing Pipeline

In [14]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude=object)),
        ("cat", categorical_transformer, selector(dtype_include=object)),
    ]
)


<1x200589 sparse matrix of type '<class 'numpy.float64'>'
	with 90 stored elements in Compressed Sparse Row format>

### Testing

In [15]:
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

testing_dat = new_dat.sample(n=25000,axis=0)

X = testing_dat.loc[:,testing_dat.columns!="Accident Type"]
Y = testing_dat.loc[:,"Accident Type"]

# creating testing and training data
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=.25)

print("X-train shape: ", x_train.shape)
print("Y-train shape: ", y_train.shape)
print("X-test shape: ", x_test.shape)
print("Y-test shape: ", y_test.shape)

# turn conf matrix in df for visualization
def conf_matrix_to_df(conf_matrix, target_names):
    return pd.DataFrame(conf_matrix, columns=target_names, index=target_names)

rf_pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

rf_pipe.fit(x_train, y_train)
pred = rf_pipe.predict(x_test)
print(rf_pipe.score(x_test,y_test))
conf_mat = confusion_matrix(y_test, pred)
conf_mat_df = conf_matrix_to_df(conf_mat,testing_dat['Accident Type'].unique())
conf_mat_df

X-train shape:  (18750, 91)
Y-train shape:  (18750,)
X-test shape:  (6250, 91)
Y-test shape:  (6250,)
0.73664


,Derailment,Other impacts,Fire/violent rupture,Other (describe in narrative),Rear end collision,Side collision,Hwy-rail crossing,Head on collision,Obstruction,Raking collision,Broken train collision,Explosion-detonation,RR grade crossing
Derailment,0,19,0,0,0,0,0,0,0,0,0,0,1
Other impacts,0,3981,0,0,0,5,0,4,37,0,0,0,22
Fire/violent rupture,0,1,0,0,0,0,0,0,0,0,0,0,1
Other (describe in narrative),0,69,0,16,0,7,0,6,1,0,0,0,6
Rear end collision,0,50,0,0,0,3,0,0,0,0,0,1,8
Side collision,0,46,0,0,0,263,0,0,0,0,0,0,0
Hwy-rail crossing,0,69,0,0,0,8,6,4,10,0,0,0,1
Head on collision,0,73,0,0,0,1,0,48,9,0,0,1,0
Obstruction,0,429,0,0,0,3,1,5,137,0,0,0,20
Raking collision,0,2,0,0,0,0,0,0,0,0,0,0,0


## Saving

In [6]:
# save data to file
data.to_csv("cleaned_data.csv",index=False) # index=False to not save the index column

In [7]:
# saves all column names to separate file in order to find different variables
with open("column_names.txt", "w") as f:
    for s in data.columns:
        f.write(s + "\n")